# Web scraping Rap genre from Spotify on April 2024


### Not including all genres web scraping. This is my notebook to share to the public.

In [ ]:
# Importing the libraries
import time
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import csv
import requests
import spotipy.util as util

- I am not sharing my API credentials because they are private

In [2]:
# Set up Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id='', client_secret='')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Function to get audio features with retry logic
def get_audio_features_with_retry(track_id, max_retries=3, delay=1, backoff=2):
    retries = 0
    while retries < max_retries:
        try:
            features = sp.audio_features(track_id)
            return features
        except spotipy.SpotifyException as e:
            if e.http_status == 429:  # Too Many Requests
                print(f"Received 429 error. Retrying after {delay} seconds...")
                time.sleep(delay)
                delay *= backoff  # Exponential backoff
                retries += 1
            else:
                raise spotipy.SpotifyException(f"Spotify API error: {e}")  # Re-raise the error with custom message

    # If retries exhausted, raise a custom error
    raise RuntimeError("Max retries reached without success.")

# Function to get only the albums I want and the respective artist (Including duos if the artists is not the only singer on the track)
def fetch_tracks(artist_album_dict, limit=50):
    """
    Fetch tracks from Spotify based on a dictionary of artists and their respective albums.
    """
    tracks_data = []
    for artist, albums in artist_album_dict.items():
        for album_name in albums:
            offset = 0
            while True:
                results = sp.search(q='artist:"{}" album:"{}"'.format(artist, album_name), limit=limit, offset=offset)
                tracks = results['tracks']['items']
                if not tracks:
                    break
                
                for track in tracks:
                    # Extract album name
                    album_name = track['album']['name']
                    
                    # Extract artists associated with the track
                    artists = [artist['name'] for artist in track['artists']]
        
                    # Check if album name is one of the specified albums and the artist is one of the specified artists for this track
                    if album_name in albums and artist in artists:
                        # Fetch audio features with retry logic
                        audio_features = get_audio_features_with_retry(track['id'])
                        if audio_features:
                            audio_features = audio_features[0]  # Only the first track will be considered
                            
                        # Fetch track info (including popularity)
                        track_info = sp.track(track['id'])
                        popularity = None
                        if track_info:
                            popularity = track_info.get('popularity')  # 'Popularity' is a general measure of the track's popularity on Spotify

                        # Add track data to the list
                        track_data = {
                            'Name': track['name'],
                            'Artists': ', '.join(artists),
                            'Album': album_name,
                            'Release Date': track['album']['release_date'],
                            'Popularity Score': popularity,  # Renamed to 'Popularity Score' for clarity
                        }
                        
                        # Add audio features as separate columns
                        if audio_features:
                            for key, value in audio_features.items():
                                track_data[key] = value
                        
                        tracks_data.append(track_data)
                
                offset += limit
    
    return tracks_data


# Define the dictionary with RAP artists and their respective albums
artist_album_dict = {
    "2Pac": ["2Pacalypse Now", "Strictly 4 My N.I.G.G.AZ…", "Me Against The World", "All Eyez On Me"],
    "Eminem": ["Infinite", "The Slim Shady LP", "The Marshall Mathers LP", "The Eminem Show",
               "Encore (Deluxe Version)", "Relapse", "Recovery", "The Marshall Mathers LP2 (Deluxe)",
               "Kamikaze", "Music To Be Murdered By"],
    "Kanye West": ["The College Dropout", "Late Registration", "Graduation", "808s & Heartbreak",
                   "My Beautiful Dark Twisted Fantasy", "Yeezus", "The Life Of Pablo", "ye",
                   "JESUS IS KING", "Donda"],
    "Drake": ["Thank Me Later", "Take Care (Deluxe)", "Nothing Was The Same", "Views", "Scorpion",
              "Certified Lover Boy", "Honestly, Nevermind", "For All The Dogs"],
    "Snoop Dogg": ["Doggystyle", "Tha Doggfather", "Da Game Is to Be Sold, Not To Be Told",
                   "No Limit Top Dogg",  "Tha Last Meal", "Paid Tha Cost To Be Da Bo$$",
                   "R&G (Rhythm & Gangsta): The Masterpiece", "Tha Blue Carpet Treatment",
                   "Ego Trippin’", "Malice ‘N Wonderland", "Doggumentary", "BUSH", "COOLAID",
                   "Neva Left",  "Snoop Dogg Presents Bible of Love", "I Wanna Thank Me",
                   "From Tha Streets 2 Tha Suites", "BODR"]
}

- Wikipedia helped me Identify the studio albums on Wikipedia, but to extract with the code, I needed the albums as they appear on Spotify.

- Since it can be difficult to distinguish studio albums on Spotify, especially due to unfamiliarity with various genres, artists and albums.

In [4]:
# Fetch tracks data
rap_df = fetch_tracks(artist_album_dict)

# Convert the list of dictionaries to a pandas DataFrame
rap_df = pd.DataFrame(rap_df)

# Print the DataFrame
print(rap_df)

                              Name                                 Artists  \
0              Brenda's Got A Baby                                    2Pac   
1                          Trapped                                    2Pac   
2                          Violent                                    2Pac   
3              I Don't Give A Fuck                                    2Pac   
4                If My Homie Calls                                    2Pac   
...                            ...                                     ...   
1298            Gotta Keep Pushing          Snoop Dogg, T.I., Sleepy Brown   
1299               It’s In The Air  Snoop Dogg, Uncle Murda, JANE HANDCOCK   
1300                         Daddy                    Snoop Dogg, Emo Trap   
1301                  Catch a Vibe                     Snoop Dogg, HeyDeon   
1302  We Don’t Gotta Worry No More                 Snoop Dogg, Wiz Khalifa   

               Album Release Date  Popularity Score  danceabili

- Fetching this tracks took 8 minutes

In [5]:
# Save the data to a CSV file if RAP_df is not empty
if not rap_df.empty:
    csv_filename = 'rap.csv'
    rap_df.to_csv(csv_filename, index=False, encoding='utf-8')
    print(f"Data has been saved to {csv_filename}")
else:
    print("No tracks found for the given search queries.")

Data has been saved to rap.csv
